#### Friday, December 20, 

This alll runs in one pass.

In [1]:
# Deliberately set the OPENAI_API_KEY to an invalid value to ensure that the code is not using it.
import os
os.environ['OPENAI_API_KEY'] = "Nope!"

In [2]:
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
)

In [3]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from pydantic.v1 import BaseModel, Field
from typing import List

In [4]:
temperature = 0.0

class BusinessName(BaseModel):
    name: str = Field(description="The name of the business")
    rating_score: float = Field(description="The rating score of the business. 0 is the worst, 10 is the best.")

In [5]:
class BusinessNames(BaseModel):
    names: List[BusinessName] = Field(description="A list of business names")

In [6]:
# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=BusinessNames)

In [7]:
principles = """
- The name must be easy to remember.
- Use the {industry} industry and Company context to create an effective name.
- The name must be easy to pronounce.
- You must only return the name without any other text or characters.
- Avoid returning full stops, \n or any other characters.
- The maximum length of the name must be 10 characters.
"""

In [8]:
# Chat Model Output Parser:
# model = ChatOpenAI() 
lmstudio = "http://localhost:1234/v1"
model = "qwen2.5-14b-instruct@q8_0" # lmstudio-community/Qwen2.5-14B-Instruct-GGUF :  Qwen2.5-14B-Instruct-Q4_K_M.gguf

chat = ChatOpenAI(base_url=lmstudio, model=model, api_key="LMStudio", temperature=temperature)

In [9]:
template = """Generate 5 business names for a new start up company in the {industry} industry. 
You must follow the following principles: {principles}
{format_instructions}
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])

In [10]:
# prompt_and_model = chat_prompt | model
prompt_and_model = chat_prompt | chat

In [11]:
result = prompt_and_model.invoke(
    {
        "principles": principles,
        "industry": "Data Science",
        "format_instructions": parser.get_format_instructions(),
    }
)
print(parser.parse(result.content))

# 21.3s

names=[BusinessName(name='DataViz', rating_score=9.0), BusinessName(name='InsightIQ', rating_score=8.5), BusinessName(name='CodeNest', rating_score=9.2), BusinessName(name='DataSpark', rating_score=8.7), BusinessName(name='BrainByte', rating_score=9.1)]


In [12]:
# chain = chat_prompt | model | parser
chain = chat_prompt | chat | parser

result = chain.invoke(
    {
        "principles": principles,   
        "industry": "Data Science",
        "format_instructions": parser.get_format_instructions(),
    }
)
print(result)

# 21.9s

names=[BusinessName(name='DataViz', rating_score=9.0), BusinessName(name='InsightIQ', rating_score=8.5), BusinessName(name='CodeNest', rating_score=9.2), BusinessName(name='DataSpark', rating_score=8.7), BusinessName(name='BrainByte', rating_score=9.1)]


In [13]:
# Technically you can also write it like model() which uses the __call__ method. However the _call__ method is an older syntax.

# Or simply use model.invoke(input) whilst using .format_prompt():
# chat_response_output = model.invoke(
#     chat_prompt.format_prompt(
#         principles=principles,
#         industry="Data Science",
#         format_instructions=parser.get_format_instructions(),
#     ).to_messages()
# )

chat_response_output = chat.invoke(
    chat_prompt.format_prompt(
        principles=principles,
        industry="Data Science",
        format_instructions=parser.get_format_instructions(),
    ).to_messages()
)

pydantic_schema: BusinessNames = parser.parse(chat_response_output.content)
print(pydantic_schema, "\n")
print(pydantic_schema.json())

# 21.6s

names=[BusinessName(name='DataViz', rating_score=9.0), BusinessName(name='InsightIQ', rating_score=8.5), BusinessName(name='CodeNest', rating_score=9.2), BusinessName(name='DataSpark', rating_score=8.7), BusinessName(name='BrainByte', rating_score=9.1)] 

{"names": [{"name": "DataViz", "rating_score": 9.0}, {"name": "InsightIQ", "rating_score": 8.5}, {"name": "CodeNest", "rating_score": 9.2}, {"name": "DataSpark", "rating_score": 8.7}, {"name": "BrainByte", "rating_score": 9.1}]}
